### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [85]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

#%matplotlib inline
#%load_ext pycodestyle_magic

In [243]:
#%%pycodestyle

class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=990, sufficient_share=1.0, criterion='gini', max_features=None):
        self.purity_index = 1
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return l_s + r_s - np.sum(l_c**2, axis=1) / l_s - np.sum(r_c**2, axis=1) / r_s
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        l_entropy = np.sum(l_c * np.log(l_c), axis=1) - l_s * np.log(l_s)
        r_entropy = np.sum(r_c * np.log(r_c), axis=1) - r_s * np.log(r_s)
        return (l_entropy + r_entropy) / 2.0
        
    def __misclass(self, l_c, l_s, r_c, r_s):
        l_misclass = 1 - l_c / np.repeat(l_s, l_c.shape[1]).reshape(-1, l_c.shape[1]) 
        r_misclass = 1 - r_c / np.repeat(r_s, r_c.shape[1]).reshape(-1, r_c.shape[1])
        return (l_misclass + r_misclass) / 2.0

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, int(feature_id)] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        x_data, y_data = self.__sort_samples(x, y)
        split_y = y_data[self.min_samples_split:-self.min_samples_split]
        critical_points_ids = np.where(split_y[1:] != split_y[:-1])[0] + self.min_samples_split + 1
        
        if len(critical_points_ids) == 0:
            return np.inf, None
        
        points_delta = critical_points_ids - np.append([self.min_samples_split], critical_points_ids[:-1])
        mas = np.zeros((critical_points_ids.shape[0], self.num_class))
        mas[np.arange(critical_points_ids.shape[0]), y_data[critical_points_ids - 1]] = 1
        cls = mas * points_delta.reshape(-1, 1)
        cls[0] = cls[0] + np.bincount(y_data[:self.min_samples_split], minlength=self.num_class)
        
        tmp = np.bincount(y_data)
        template = np.hstack((tmp, np.zeros(self.num_class - tmp.shape[0])))  
        l_c = np.cumsum(cls, axis=0)  
        r_c = template - l_c
        l_s = critical_points_ids.reshape(l_c.shape[0])
        r_s = y_data.shape[0] - l_s

        g = self.G_function(l_c, l_s, r_c, r_s)
        id_x = np.argmin(g)
    
        elem_id = l_s[id_x]
        return g[id_x], (x_data[elem_id-1] + x_data[elem_id]) / 2.0
            

    def __fit_node(self, x, y, node_id, depth):
        
        if ((self.max_depth <= depth) or \
            (self.sufficient_share <= sum((np.unique(y, return_counts=True)[1] / y.shape[0])**2)) or \
            (y.shape[0] < self.min_samples_split)):
            
            self.tree[node_id] = [self.__class__.LEAF_TYPE, np.argmax(np.bincount(y))]
            
        else:
            features = self.get_feature_ids(x.shape[1])
            mas = np.array([self.__find_threshold(x[:, f], y) for f in features])
            best_feature = np.argmin(mas[:,0])
            best_threshold = mas[best_feature, 1]
            
            if best_threshold == None:
                self.purity_index *= sum((np.unique(y, return_counts=True)[1] / y.shape[0])**2)
                self.tree[node_id] = [self.__class__.LEAF_TYPE, np.argmax(np.bincount(y))] 
            else:
                l_x, r_x, l_y, r_y = self.__div_samples(x, y, best_feature, best_threshold)
                if (l_x.size == 0) or (r_x.size == 0):
                    self.tree[node_id] = [self.__class__.LEAF_TYPE, np.argmax(np.bincount(y))]
                else:   
                    self.tree[node_id] = [self.__class__.NON_LEAF_TYPE, best_feature, best_threshold]
                    self.__fit_node(l_x, l_y, 2*node_id + 1, depth+1)
                    self.__fit_node(r_x, r_y, 2*node_id + 2, depth+1)
        pass
    
    def fit(self, x, y):
        self.num_class = np.unique(y).shape[0]
        self.dim_X = x.shape[1]
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [343]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [344]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [345]:
%time clf.fit(X_train, y_train)

Wall time: 2 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [346]:
%time my_clf.fit(X_train, y_train)

Wall time: 21.9 ms


# Проверка качества работы на wine

In [347]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8857142857142858

In [348]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8857142857142858

# Подготовка данных Speed Dating Data

In [355]:
df = pd.read_csv('data/Speed Dating Data.csv', encoding='cp1251') 

for i in df.columns:
    if type(df[i][0]) not in [np.float64, np.int64, float]:
        df.drop(i,axis=1, inplace=True)

for i in df.columns:
    if df[i].isnull().sum()>500:
        df.drop(i,axis=1, inplace=True)

df = df.fillna(-10000)
df.drop('id', axis=1)
y = df.loc[:, 'gender'].values
X = df.drop('gender', axis=1).iloc[:,:].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [356]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

# Проверка скорости работы на Speed Dating Data 

In [357]:
%time clf.fit(X_train, y_train)

Wall time: 121 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [358]:
%time my_clf.fit(X_train, y_train)

Wall time: 1.18 s


## Проверка качества работы на Speed Dating Data

In [359]:
%time f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

Wall time: 2 ms


0.9880662139992709

In [360]:
%time f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

Wall time: 13 ms


0.9695690948802982

## Задание 3

In [361]:
def calc_feature_importances(my_clf):
    
    def recur_calc_feature_importances(node_id, feature_importances, count): 
        node = my_clf.tree[node_id]
        if node[0] == my_clf.__class__.NON_LEAF_TYPE:
            _, feature_id, _ = node
            feature_importances[feature_id] += 1
            count += 1
            
            _, tmp = recur_calc_feature_importances(2 * node_id + 1, feature_importances, 0)
            count += tmp
            _, tmp = recur_calc_feature_importances(2 * node_id + 2, feature_importances, 0)
            count += tmp
        
        return feature_importances, count
    
    feature_importances = np.zeros(my_clf.dim_X)
    feature_importances, count = recur_calc_feature_importances(0, feature_importances, 0)
    feature_importances /= count
    
    return feature_importances

## Задание 4

In [368]:
my_feature_list = calc_feature_importances(my_clf)
my_idx_list = my_feature_list.argsort()[-10:][::-1]
my_res = [[idx, my_feature_list[idx]] for idx in my_idx_list]

feature_list = clf.feature_importances_
idx_list = feature_list.argsort()[-10:][::-1]
res = [[idx, feature_list[idx]] for idx in idx_list]

print("My:                   Skearn:")
print("id  importance        id  importance")
for i in range(len(res)):
    print('{:2}'.format(my_res[i][0]), ' {0:10e}'.format(my_res[i][1]), '     {:2}'.format(res[i][0]), ' {0:10e}'.format(res[i][1]))

My:                   Skearn:
id  importance        id  importance
 0  9.401709e-02      56  2.741704e-01
56  5.982906e-02      62  2.073823e-01
 1  4.273504e-02      15  1.037725e-01
35  4.273504e-02       0  3.920737e-02
60  4.273504e-02      53  2.503208e-02
66  3.418803e-02       9  2.466296e-02
 9  3.418803e-02      60  2.372600e-02
62  3.418803e-02      33  2.189670e-02
17  3.418803e-02      68  1.950069e-02
49  2.564103e-02      48  1.903870e-02


In [369]:
print("All feature importance\n")
print("My:", "\n", calc_feature_importances(my_clf), '\n\n')
print("Sklearn:", "\n", clf.feature_importances_)

All feature importance

My: 
 [0.09401709 0.04273504 0.00854701 0.         0.00854701 0.01709402
 0.02564103 0.         0.         0.03418803 0.         0.00854701
 0.00854701 0.00854701 0.         0.02564103 0.         0.03418803
 0.00854701 0.01709402 0.         0.00854701 0.         0.
 0.         0.         0.         0.         0.         0.00854701
 0.01709402 0.00854701 0.01709402 0.02564103 0.00854701 0.04273504
 0.02564103 0.00854701 0.00854701 0.00854701 0.         0.
 0.00854701 0.01709402 0.         0.01709402 0.01709402 0.
 0.00854701 0.02564103 0.         0.00854701 0.         0.02564103
 0.00854701 0.02564103 0.05982906 0.01709402 0.         0.00854701
 0.04273504 0.00854701 0.03418803 0.00854701 0.01709402 0.00854701
 0.03418803 0.         0.02564103 0.         0.00854701 0.00854701
 0.01709402 0.         0.         0.         0.         0.
 0.         0.00854701 0.        ] 


Sklearn: 
 [0.03920737 0.00311386 0.00518803 0.         0.00159076 0.01785454
 0.00053716 0. 

## Задание 5

In [161]:
clf = RandomForestClassifier(n_estimators=50)
param_dist = {
              "min_samples_split": range(2, 5),
              "criterion": ["gini", "entropy"],
              "max_features": range(4, 10)
            }

In [162]:
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=20, cv=5)
grid_search = GridSearchCV(clf, param_grid=param_dist, cv=5)

In [163]:
%time random_search.fit(X_train, y_train)

Wall time: 37.1 s


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=20, n_jobs=None,
          param_distributions={'min_samples_split': range(2, 5), 'criterion': ['gini', 'entropy'], 'max_features': range(4, 10)},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [164]:
%time grid_search.fit(X_train, y_train)

Wall time: 1min 4s


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'min_samples_split': range(2, 5), 'criterion': ['gini', 'entropy'], 'max_features': range(4, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [165]:
f1_score(y_pred=random_search.predict(X_test), y_true=y_test, average='macro')

1.0

In [166]:
f1_score(y_pred=grid_search.predict(X_test), y_true=y_test, average='macro')

1.0